In [29]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

np.set_printoptions(6, suppress=True)

plt.style.use([
    'grid',
    'notebook'
])

In [30]:
A_vertices = []
B_vertices = []

Ts = .1

r = .31
mu = .4
sigma = .55
A2 = np.pi * r**2

Kb = 15.92e-4

In [31]:
h1_interval = np.linspace(20, 74, 1000)
h2_interval = np.linspace(12, 50, 1000)

Z1 = []
Z2 = []
Z3 = []

c = 0
min_diff = 6

for i in range(len(h1_interval)):
    h1 = h1_interval[i]
    
    for j in range(i, len(h1_interval)): 
        h2 = h2_interval[j]

        diff = h1 - h2

        if diff < min_diff:
            continue

        R12 = (0.2371*diff+6.9192)*10
        q0  = (18.6367*h2+700.6831)*1e-4
        a1  = (3*r/5)*(2.7*r-((np.cos(2.5*np.pi*(h1-8)*1e-2-mu))/(sigma*np.sqrt(2*np.pi)))*np.exp(-(((h1-8)*1e-2-mu)**2)/(2*sigma**2)))

        z1 = 1/R12
        z2 = q0/h2
        z3 = 1/a1

        Z1.append(z1)
        Z2.append(z2)
        Z3.append(z3)


min_z1, max_z1 = min(Z1), max(Z1)
min_z2, max_z2 = min(Z2), max(Z2)
min_z3, max_z3 = min(Z3), max(Z3)

print(min_z1, max_z1)
print(min_z2, max_z2)
print(min_z3, max_z3)

0.00793046567694455 0.01198782037449951
0.0032650362 0.007702695833333333
3.4476944509493617 18.456983136119643


In [32]:
z1_bounds = np.array([-.5160, 2.3060])
z2_bounds = np.array([.0033, .0077])
z3_bounds = np.array([3.4478, 18.4570])

# z1_bounds = np.array([min_z1, max_z1])
# z2_bounds = np.array([min_z2, max_z2])
# z3_bounds = np.array([min_z3, max_z3])

for z1 in z1_bounds:
    for z2 in z2_bounds:
        for z3 in z3_bounds:
            Ai = np.array([
                [-z1*z3, z1*z3],
                [z1/A2, (-z1-z2)/A2]
            ])

            Bi = np.array([
                [Kb*z3],
                [.0]
            ])

            print(
                np.linalg.eigvals(Ai)
            )

            A_vertices.append(Ai)
            B_vertices.append(Bi)

[ 3.482853 -0.005583]
[11.231286 -0.009269]
[ 3.47575  -0.013055]
[11.229074 -0.021631]
[-15.594097  -0.005573]
[-50.201618  -0.009267]
[-15.601246  -0.012997]
[-50.203837  -0.021622]


In [33]:
Z1_, Z2_, Z3_ = sp.symbols('Z1 Z2 Z3')

z1_min, z1_max = z1_bounds
z2_min, z2_max = z2_bounds
z3_min, z3_max = z3_bounds

M1 = (z1_max - Z1_) / (z1_max - z1_min)
N1 = (z2_max - Z2_) / (z2_max - z2_min)
P1 = (z3_max - Z3_) / (z3_max - z3_min)

M2 = 1 - M1
N2 = 1 - N1
P2 = 1 - P1

In [34]:
print(' ')
print(M1)
print(N1)
print(P1)
print(' ')
print(M2)
print(N2)
print(P2)

 
0.817150956768249 - 0.354358610914245*Z1
1.75 - 227.272727272727*Z2
1.22971244303494 - 0.0666258028409242*Z3
 
0.354358610914245*Z1 + 0.182849043231751
227.272727272727*Z2 - 0.75
0.0666258028409242*Z3 - 0.229712443034939


In [35]:
for i in range(len(A_vertices)):
    A = A_vertices[i]
    B = B_vertices[i]
    
    np.save(f'./data/vertices/A_{i}', A)
    np.save(f'./data/vertices/B_{i}', B)